# Preparação

In [ ]:
pip install python-sat

     |████████████████████████████████| 1.8MB 6.6MB/s 


Arestas do grafo que iremos procurar uma solução.

In [ ]:
# altere a variável a baixo.
arestas = ["AB", "AF", "AG", "BC", "CG", "CE", "CD", "DE", "EG", "EF"]

Cria variáveis auxiliares.

In [ ]:
formulas = []
predicados = []

Garante que tem as arestas inseridas nos dois entidos, remove repetidos e calcula a quantidade de passos.

In [ ]:
arestas.extend([a[::-1] for a in arestas])
arestas = list(set(arestas))
qtd_passos = int(len(arestas) / 2)

Gera a matriz com todos os possíveis predicados.

In [ ]:
for i, aresta in enumerate(arestas):
  for j in range(qtd_passos):
    predicados.append(f"{j+1}_{aresta}")

# Criação de regras

### Restrições de duplicidade

Garante que uma posição terá somente uma aresta.

In [ ]:
for atual in predicados:
  pos_atual = atual.split('_')[0]
  for outro in predicados:
    pos_outro = outro.split('_')[0]
    if atual != outro and pos_atual == pos_outro:
      formulas.append([f"¬{atual}",f"¬{outro}"])

Garante que uma aresta será utilizada apenas em uma posição.

In [ ]:
for atual in predicados:
  aresta_atual = atual.split('_')[1]
  for outro in predicados:
    aresta_outro = outro.split('_')[1]
    if atual != outro and aresta_atual == aresta_outro:
      formulas.append([f"¬{atual}",f"¬{outro}"])

### Restrição de transição

Gera as fórmulas que formam os possíveis próximos passos.

In [ ]:
for atual in predicados:
  aux = [f"¬{atual}"]
  pos_atual, aresta_atual = atual.split('_')
  for outro in predicados:
    pos_outro, aresta_outro = outro.split('_')
    if atual != outro:
      if int(pos_atual)+1 == int(pos_outro) and aresta_atual[-1] == aresta_outro[-2] and aresta_atual[0] != aresta_outro[1]:
        aux.append(outro)
  if len(aux) > 1:
    formulas.append(aux)

### Restrição de volta
Impedem a utilização de uma aresta se esta já foi utilizada no sentido contrário.

In [ ]:
for atual in predicados:
  aresta_atual = atual.split('_')[1]
  for outro in predicados:
    aresta_outro = outro.split('_')[1]
    if aresta_atual == aresta_outro[::-1]:
      formulas.append([f"¬{atual}", f"¬{outro}"])

### Restrição de utilização das arestas

Garante que todas as arestas serão utilizadas.

In [ ]:
_aux = dict()

for predicado in predicados:
  aresta = predicado.split('_')[1]

  if aresta in _aux:
    _aux[aresta].append(predicado)
  elif aresta[::-1] in _aux:
    _aux[aresta[::-1]].append(predicado)
  else:
    _aux[aresta] = [predicado]

for key in _aux:
  formulas.append(_aux[key]) 

# Resolução

Prepara o python-sat e Glucose.

In [ ]:
from pysat.solvers import Glucose4
from pysat.formula import CNF
from pysat.solvers import Solver

g = Glucose4()
formula = CNF()

Transforma e adiciona as formulas geradas.

In [ ]:
for _formula in formulas:
  aux = list()
  for predicado in _formula:
    if predicado in predicados:
      aux.append(predicados.index(predicado) + 1)
    elif '¬' == predicado[0] and predicado[1:] in predicados:
      aux.append((predicados.index(predicado[1:]) +1 ) * -1)
  formula.append(aux)

g.append_formula(formula)

Tenta solucionar.

In [ ]:
if g.solve():
  resposta = sorted([predicados[x-1] for x in g.get_model() if x>0], key=lambda x: int(x.split('_')[0]))
  print("Resposta:", resposta)
else:
  print("É insatisfazível.")

Resposta: ['1_GE', '2_EC', '3_CB', '4_BA', '5_AG', '6_GC', '7_CD', '8_DE', '9_EF', '10_FA']
